In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd
#import regex module
import re 




In [4]:
driver = webdriver.Chrome('chromedriver')
#This line creates a ChromeOptions object, which allows you to customize the behavior of the ChromeDriver.
options=webdriver.ChromeOptions()

#This is useful for running automated tests or performing web scraping tasks without displaying the browser window.
#options.add_argument('-headless')

#disable chrome contenairiation in order to modify options
options.add_argument('-no-sandbox')

#disable shared memory space in Chrome cause might be troublesome in headless mode
options.add_argument('-disable-dev-shm-usage')

#ignores SSL certificate errors
options.add_argument('--ignore-certificate-errors')

#pass in incognito mode
#options.addArguments("incognito");

#pass arguments
driver = webdriver.Chrome('chromedriver',options=options)



In [9]:
#Navigate to page
driver.get('https://www.enseignement-prive.info/annuaire-enseignement-prive/ile-de-france')

Etablissements =  driver.find_elements(By.CSS_SELECTOR,'body > section > div.wrap > section > div.clearfix.establishments-list > article:nth-child(n) > header > strong > a')
articles=[]                                            
i=1                                                    

for Etablissement in Etablissements :
    #Next rank if i divisible by 11
    if i%11 == 0:
        i+=1  
    #Indent url etablissement CSS selector
    IndentedEtablissementLinkSelector = f'body > section > div.wrap > section > div.clearfix.establishments-list > article:nth-child({i}) > header > strong > a'
    #Wait
    element = WebDriverWait(driver, 45).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, IndentedEtablissementLinkSelector))
    ) 
    #find url container                
    UrlEtablissementContainer = driver.find_element(By.CSS_SELECTOR,IndentedEtablissementLinkSelector)                             
    #retrieve  NomEtablissementScolaire
    NomEtablissementScolaire = UrlEtablissementContainer.text
    #Get url
    url = f"{UrlEtablissementContainer.get_attribute('href')}"
    #Navigate to fiche etablissement
    driver.get(url)
    #Wait for adress to be located
    element = WebDriverWait(driver, 45).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'address'))
    )
    Adresse = driver.find_element(By.CLASS_NAME, 'address').text
    WebDriverWait(driver, 45).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'phone-toggle'))
    )
    TelephoneAccess = driver.find_element(By.CLASS_NAME, 'phone-toggle')
    TelephoneAccess.click()
    WebDriverWait(driver, 45).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'body > section > div.wrap > section > article > header > div.coords > div > a'))
        )         
    Telephone = driver.find_element(By.CSS_SELECTOR, 'body > section > div.wrap > section > article > header > div.coords > div > a').text
    articles.append([NomEtablissementScolaire, Adresse, Telephone])
    driver.back()
    i+=1
df = pd.DataFrame(articles ,columns = [NomEtablissementScolaire, Adresse, Telephone])


In [10]:
df.info()
df.iloc[:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column                                                                              Non-Null Count  Dtype 
---  ------                                                                              --------------  ----- 
 0   Apprentis d'Auteuil - Lycée Professionnel et UFA Apprentissage Poullart des Places  30 non-null     object
 1   3 boulevard de Stalingrad
94320 Thiais                                              30 non-null     object
 2   01 58 42 75 92                                                                      30 non-null     object
dtypes: object(3)
memory usage: 852.0+ bytes


,Apprentis d'Auteuil - Lycée Professionnel et UFA Apprentissage Poullart des Places,3 boulevard de Stalingrad\n94320 Thiais,01 58 42 75 92
0,Apprentis d'Auteuil - Collège Saint-Gabriel,21 rue de la Lisette\n92220 Bagneux,01 46 57 61 22
1,Lycée Général et Technologique Privé Saint- Je...,31 rue Caulaincourt\n75018 Paris 18,01 46 06 03 08
2,Collège Privé Mixte Mercier Saint-Paul,1 rue des Annonciades\n78250 Meulan,01 34 92 84 20
3,Apprentis d'Auteuil - Ensemble Scolaire Saint-...,1 rue du Père Brottier\n92190 Meudon,01 46 23 62 81
4,Collège Privé Saint-Charles,55 rue Gabriel-Péri\n95240 Cormeilles-en-Parisis,01 39 78 03 41
5,Collège Privé Jeanne d'Arc,44 rue Gabriel Péri\n92120 Montrouge,01 42 53 18 95
6,Apprentis d'Auteuil - Lycée Saint-Gabriel,21 rue de la Lisette\n92220 Bagneux,01 46 57 61 22
7,Lycée Agricole Privé Sully,22 avenue de l'Europe\n78200 Magnanville,01 34 77 27 00
8,Lycée Privé Sainte-Élisabeth,112 rue de Lourmel\n75015 Paris 15,01 45 54 56 76
9,Lycée Polyvalent Privé Albert de Mun,,01 43 06 33 09


In [ ]:
GrossAdresse = driver.find_element(By.CLASS_NAME, 'address').text
    pattern = r'\n'
    CleanAdresse = re.sub(pattern,'',GrossAdresse)
    articles.append([NomEtablissementScolaire, GrossAdresse ])

In [55]:
df.iloc[:]

,CPGE - Ensemble Scolaire Saint Michel de Picpus,
0,Collège Privé Saint-Charles Notre-Dame,
1,Guidepost Montessori Paris Monceau,93 rue de Monceau\n75008 Paris 08
2,École Privée Saint-Joseph,
3,Collège Privé Catholique La Salle Notre-Dame d...,
4,Collège Privé - Groupe Scolaire Sainte-Jeanne ...,
5,École Privée Catholique Saint Bernard - Sainte...,
6,Collège Privé Notre-Dame Les Oiseaux,
7,École Privée Saint Jean Hulst,
8,Lycée Privé Saint-Louis Saint-Clément,
9,"Lycée Professionnel des métiers du commerce, d...",
